In [1]:
pip install line-bot-sdk

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
from flask import Flask, request, abort
from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
    QuickReply,
    QuickReplyItem,
    MessageAction
)
from linebot.v3.webhooks import MessageEvent, TextMessageContent

import pandas as pd
import os

# 讀取環境變數
line_access_token = os.environ.get('LINE_ACCESS_TOKEN')
line_secret = os.environ.get('LINE_SECRET')

# 初始化 Flask & LINE
app = Flask(__name__)
configuration = Configuration(access_token=line_access_token)
handler = WebhookHandler(line_secret)

# 載入資料
hotel_data = pd.read_csv('./Hotel_E_F.csv')
food_data = pd.read_csv('./Taiwan_Food_5000.csv')
attraction_data = pd.read_csv('./Taiwan_Attractions_5000.csv')

# 使用者選擇紀錄
user_city = {}

# 縣市代碼（for 即時天氣）
cities = {
    "台北": "63", "新北": "65", "基隆": "10017", "桃園": "68", "新竹市": "10018", "新竹縣": "10004",
    "苗栗": "10005", "台中": "66", "彰化": "10007", "南投": "10008", "雲林": "10009", "嘉義市": "10020",
    "嘉義縣": "10010", "台南": "67", "高雄": "64", "屏東": "10013", "宜蘭": "10002", "花蓮": "10015",
    "台東": "10014", "澎湖": "10016", "金門": "09020", "連江": "09007"
}

# 分頁城市選單
city_pages = [
    ["台北", "新北", "基隆", "桃園", "新竹市", "新竹縣", "苗栗", "台中"],
    ["彰化", "南投", "雲林", "嘉義市", "嘉義縣", "台南", "高雄", "屏東"],
    ["宜蘭", "花蓮", "台東", "澎湖", "金門", "連江"]
]

# district_options 請貼你前面那段完整行政區字典在這裡
district_options = {
    "台北": ["中正區", "大同區", "中山區", "松山區", "大安區", "萬華區", "信義區", "士林區", "北投區", "內湖區", "南港區", "文山區"],
    "新北": ["板橋區", "三重區", "中和區", "永和區", "新莊區", "新店區", "土城區", "蘆洲區", "汐止區", "淡水區", "林口區", "三峽區", "鶯歌區", "五股區", "泰山區", "樹林區", "瑞芳區", "八里區", "深坑區", "石碇區", "坪林區", "三芝區", "石門區", "平溪區", "雙溪區", "貢寮區"],
    "基隆": ["仁愛區", "信義區", "中正區", "中山區", "安樂區", "暖暖區", "七堵區"],
    "桃園": ["桃園區", "中壢區", "平鎮區", "八德區", "楊梅區", "蘆竹區", "大溪區", "大園區", "龜山區", "龍潭區", "新屋區", "觀音區", "復興區"],
    "新竹市": ["東區", "北區", "香山區"],
    "新竹縣": ["竹北市", "竹東鎮", "新埔鎮", "關西鎮", "湖口鄉", "新豐鄉", "芎林鄉", "寶山鄉", "北埔鄉", "峨眉鄉", "橫山鄉", "尖石鄉", "五峰鄉"],
    "苗栗": ["苗栗市", "頭份市", "苑裡鎮", "通霄鎮", "竹南鎮", "後龍鎮", "卓蘭鎮", "西湖鄉", "頭屋鄉", "公館鄉", "銅鑼鄉", "三義鄉", "造橋鄉", "三灣鄉", "南庄鄉", "大湖鄉", "獅潭鄉", "泰安鄉"],
    "台中": ["中區", "東區", "南區", "西區", "北區", "西屯區", "南屯區", "北屯區", "豐原區", "大里區", "太平區", "清水區", "沙鹿區", "龍井區", "大雅區", "霧峰區", "烏日區", "后里區", "東勢區", "神岡區", "大肚區", "外埔區", "大安區", "新社區", "石岡區", "和平區"],
    "彰化": ["彰化市", "鹿港鎮", "和美鎮", "員林市", "溪湖鎮", "田中鎮", "北斗鎮", "二林鎮", "線西鄉", "伸港鄉", "福興鄉", "秀水鄉", "花壇鄉", "芬園鄉", "大村鄉", "埔鹽鄉", "埔心鄉", "永靖鄉", "社頭鄉", "二水鄉", "田尾鄉", "埤頭鄉", "溪州鄉", "竹塘鄉", "大城鄉", "芳苑鄉"],
    "南投": ["南投市", "埔里鎮", "草屯鎮", "竹山鎮", "集集鎮", "名間鄉", "鹿谷鄉", "中寮鄉", "魚池鄉", "國姓鄉", "水里鄉", "信義鄉", "仁愛鄉"],
    "雲林": ["斗六市", "斗南鎮", "虎尾鎮", "西螺鎮", "土庫鎮", "北港鎮", "莿桐鄉", "林內鄉", "二崙鄉", "崙背鄉", "麥寮鄉", "東勢鄉", "褒忠鄉", "臺西鄉", "元長鄉", "四湖鄉", "口湖鄉", "水林鄉"],
    "嘉義市": ["東區", "西區"],
    "嘉義縣": ["太保市", "朴子市", "布袋鎮", "大林鎮", "民雄鄉", "溪口鄉", "新港鄉", "六腳鄉", "東石鄉", "義竹鄉", "鹿草鄉", "水上鄉", "中埔鄉", "竹崎鄉", "梅山鄉", "番路鄉", "大埔鄉", "阿里山鄉"],
     "台南": ["中西區", "東區", "南區", "北區", "安平區", "安南區", "永康區", "歸仁區", "新化區", "左鎮區", "玉井區", "楠西區", "南化區", "仁德區", "關廟區", "龍崎區", "官田區", "麻豆區", "佳里區", "西港區", "七股區", "將軍區", "學甲區", "北門區", "新營區", "後壁區", "白河區", "東山區", "六甲區", "下營區", "柳營區", "鹽水區", "善化區", "大內區", "山上區", "新市區", "安定區"],
    "高雄": ["楠梓區", "左營區", "鼓山區", "三民區", "苓雅區", "新興區", "前金區", "鹽埕區", "前鎮區", "旗津區", "小港區", "鳳山區", "林園區", "大寮區", "大樹區", "大社區", "仁武區", "鳥松區", "岡山區", "橋頭區", "燕巢區", "田寮區", "阿蓮區", "路竹區", "湖內區", "茄萣區", "永安區", "彌陀區", "梓官區", "旗山區", "美濃區", "六龜區", "甲仙區", "杉林區", "內門區", "茂林區", "桃源區", "那瑪夏區"],
    "屏東": ["屏東市", "潮州鎮", "東港鎮", "恆春鎮", "萬丹鄉", "長治鄉", "麟洛鄉", "九如鄉", "里港鄉", "鹽埔鄉", "高樹鄉", "萬巒鄉", "內埔鄉", "竹田鄉", "新埤鄉", "枋寮鄉", "新園鄉", "崁頂鄉", "林邊鄉", "南州鄉", "佳冬鄉", "琉球鄉", "車城鄉", "滿州鄉", "枋山鄉", "三地門鄉", "霧台鄉", "瑪家鄉", "泰武鄉", "來義鄉", "春日鄉", "獅子鄉", "牡丹鄉"],
    "宜蘭": ["宜蘭市", "羅東鎮", "蘇澳鎮", "頭城鎮", "礁溪鄉", "壯圍鄉", "員山鄉", "冬山鄉", "五結鄉", "三星鄉", "大同鄉", "南澳鄉"],
    "花蓮": ["花蓮市", "鳳林鎮", "玉里鎮", "新城鄉", "吉安鄉", "壽豐鄉", "光復鄉", "豐濱鄉", "瑞穗鄉", "萬榮鄉", "秀林鄉", "卓溪鄉"],
    "台東": ["台東市", "成功鎮", "關山鎮", "卑南鄉", "大武鄉", "太麻里鄉", "東河鄉", "長濱鄉", "鹿野鄉", "池上鄉", "綠島鄉", "延平鄉", "金峰鄉", "海端鄉", "達仁鄉", "蘭嶼鄉"],
    "澎湖": ["馬公市", "湖西鄉", "白沙鄉", "西嶼鄉", "望安鄉", "七美鄉"],
    "金門": ["金城鎮", "金湖鎮", "金沙鎮", "金寧鄉", "烈嶼鄉", "烏坵鄉"],
    "連江": ["南竿鄉", "北竿鄉", "莒光鄉", "東引鄉"]
}

# 建立城市 QuickReply
def build_city_quick_reply(page=0):
    items = [QuickReplyItem(action=MessageAction(label=city, text=city)) for city in city_pages[page]]
    if page > 0:
        items.append(QuickReplyItem(action=MessageAction(label="⬅ 上一頁", text=f"城市頁 {page - 1}")))
    if page < len(city_pages) - 1:
        items.append(QuickReplyItem(action=MessageAction(label="➡ 下一頁", text=f"城市頁 {page + 1}")))
    return QuickReply(items=items)

# 建立行政區 QuickReply
def build_district_quick_reply(city, page=0, per_page=10):
    districts = district_options[city]
    start = page * per_page
    end = start + per_page
    current = districts[start:end]
    items = [QuickReplyItem(action=MessageAction(label=d, text=d)) for d in current]
    if start > 0:
        items.append(QuickReplyItem(action=MessageAction(label="⬅ 上一頁", text=f"{city}區頁 {page - 1}")))
    if end < len(districts):
        items.append(QuickReplyItem(action=MessageAction(label="➡ 下一頁", text=f"{city}區頁 {page + 1}")))
    return QuickReply(items=items)

# LINE Webhook
@app.route("/", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    except Exception as e:
        print("Error:", e)
        abort(400)
    return 'OK'

# 主處理邏輯
@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    user_message = event.message.text.strip()
    user_id = event.source.user_id

    with ApiClient(configuration) as api_client:
        bot = MessagingApi(api_client)

        if user_message == "選擇城市":
            quick = build_city_quick_reply(page=0)
            bot.reply_message(
                ReplyMessageRequest(reply_token=event.reply_token, messages=[
                    TextMessage(text="請選擇縣市：", quick_reply=quick)
                ])
            )
            return

        if user_message.startswith("城市頁 "):
            page = int(user_message.split(" ")[1])
            quick = build_city_quick_reply(page)
            bot.reply_message(
                ReplyMessageRequest(reply_token=event.reply_token, messages=[
                    TextMessage(text="請選擇縣市：", quick_reply=quick)
                ])
            )
            return

        if user_message in district_options:
            user_city[user_id] = {"city": user_message, "district": None}
            quick = build_district_quick_reply(user_message, page=0)
            bot.reply_message(
                ReplyMessageRequest(reply_token=event.reply_token, messages=[
                    TextMessage(text=f"請選擇 {user_message} 的地區（第 1 頁）：", quick_reply=quick)
                ])
            )
            return

        if "區頁" in user_message:
            city, page = user_message.split("區頁")
            city = city.strip()
            page = int(page.strip())
            quick = build_district_quick_reply(city, page)
            bot.reply_message(
                ReplyMessageRequest(reply_token=event.reply_token, messages=[
                    TextMessage(text=f"請選擇 {city} 的地區（第 {page+1} 頁）：", quick_reply=quick)
                ])
            )
            return

        if user_id in user_city and user_city[user_id]["district"] is None:
            user_city[user_id]["district"] = user_message
            city = user_city[user_id]["city"]
            district = user_city[user_id]["district"]
            bot.reply_message(
                ReplyMessageRequest(reply_token=event.reply_token, messages=[
                    TextMessage(text=f"✅ 您已選擇：{city} {district}\n請輸入「景點推薦」「美食推薦」「住宿資訊」或「即時資訊」。")
                ])
            )
            return

        # 以下是查詢功能指令
        if user_message == "指令說明":
            reply_text = (
                "【指令清單】\n"
                "1. 選擇城市\n2. 景點推薦\n3. 美食推薦\n4. 住宿資訊\n5. 即時資訊\n6. 指令說明"
            )
        elif user_message == "即時資訊":
            if user_id in user_city:
                code = cities[user_city[user_id]['city']]
                reply_text = f"{user_city[user_id]['city']} {user_city[user_id]['district']} 的天氣：\nhttps://www.cwa.gov.tw/V8/C/W/County/County.html?CID={code}"
            else:
                reply_text = "請先選擇城市與地區。"
        elif user_message == "住宿資訊":
            if user_id in user_city:
                d = user_city[user_id]['district']
                data = hotel_data[hotel_data['城市'] == d]
                reply_text = "【住宿資訊】\n" + "\n".join([
                    f"🏨 {row['旅館名稱']}\n📞 {row['電話']}\n📍 {row['地址']}\n💰 {row['最低房價']}~{row['最高房價']}\n---"
                    for _, row in data.iterrows()
                ]) if not data.empty else f"{d} 無住宿資料。"
            else:
                reply_text = "請先選擇城市與地區。"
        elif user_message == "美食推薦":
            if user_id in user_city:
                d = user_city[user_id]['district']
                data = food_data[food_data['Town'] == d]
                reply_text = "【美食推薦】\n" + "\n".join([
                    f"🥢 {row['Name_c']} ({row['Type']})\n📞 {row['Phone']}\n📍 {row['Address']}\n💰 NT${row['Price_Min']}-{row['Price_Max']}\n---"
                    for _, row in data.iterrows()
                ]) if not data.empty else f"{d} 無美食資料。"
            else:
                reply_text = "請先選擇城市與地區。"
        elif user_message == "景點推薦":
            if user_id in user_city:
                d = user_city[user_id]['district']
                data = attraction_data[attraction_data['Town'] == d]
                reply_text = "【景點推薦】\n" + "\n".join([
                    f"🏞️ {row['Name_c']}\n📞 {row['Phone']}\n📍 {row['Address']}\n---"
                    for _, row in data.iterrows()
                ]) if not data.empty else f"{d} 無景點資料。"
            else:
                reply_text = "請先選擇城市與地區。"
        else:
            reply_text = "請輸入「選擇城市」「指令說明」等指令開始使用。"

        # 最後統一回應
        bot.reply_message(
            ReplyMessageRequest(reply_token=event.reply_token, messages=[
                TextMessage(text=reply_text)
            ])
        )

if __name__ == "__main__":
    app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [07/Jun/2025 21:28:42] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [07/Jun/2025 21:28:49] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [07/Jun/2025 21:28:53] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [07/Jun/2025 21:28:59] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [07/Jun/2025 21:29:03] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [07/Jun/2025 21:29:08] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [07/Jun/2025 21:29:11] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [07/Jun/2025 21:29:12] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [07/Jun/2025 21:29:14] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [07/Jun/2025 21:29:16] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [07/Jun/2025 21:29:19] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [07/Jun/2025 21:29:23] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [07/Jun/2025 21:29:30] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [07/Jun/2025 21:29:35] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [07/Jun/2025 21:29:46] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [07/Jun/2025 2